#### Names of people in the group

Please write the names of the people in your group in the next cell.

Jacob Ørmen Clements

In [0]:
# Deleting tables left from previous runs in case they still exist after deleting an inactive cluster
dbutils.fs.rm("/user", recurse=True)

True

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
# Loading PySpark modules that we need
import unittest
from collections import Counter
from pyspark.sql import DataFrame
from pyspark.sql.types import *

#### Subtask 1: defining the schema for the data
Typically, the first thing to do before loading the data into a Spark cluster is to define the schema for the data. Look at the schema for 'badges' and try to define the schema for other tables similarly.

In [0]:
# Defining a schema for 'badges' table
badges_schema = StructType([StructField('UserId', IntegerType(), False),
                            StructField('Name', StringType(), False),
                            StructField('Date', TimestampType(), False),
                            StructField('Class', IntegerType(), False)])

#Most rows are trivially nullable and thus I will only comment the rows that are non-nullable                          

# Schema for comments table
comments_schema =StructType([
    StructField("PostId", IntegerType(), False), #Each comment must be associated with a post
    StructField("Score", IntegerType(), True),
    StructField("Text", StringType(), True),
    StructField("CreationDate", TimestampType(), False), #Each comment would have a timestamp of when it was created
    StructField("UserId", IntegerType(), True),
])



# Schema for posts table
posts_schema = StructType([
    StructField("Id", IntegerType(), False), # A unique ID for each post, can thus not have null value
    StructField("ParentId", IntegerType(), True),
    StructField("PostTypeId", IntegerType(), False), # Post must be in the form of question(1) or answer(2)
    StructField("CreationDate", TimestampType(), False), # Timestamp of when it was created
    StructField("Score", IntegerType(), True),
    StructField("ViewCount", IntegerType(), True),
    StructField("Body", StringType(), True), 
    StructField("OwnerUserId", IntegerType(), True),
    StructField("LastActivityDate", TimestampType(), True),
    StructField("Title", StringType(), True),
    StructField("Tags", StringType(), True),
    StructField("AnswerCount", IntegerType(), True),
    StructField("CommentCount", IntegerType(), True),
    StructField("FavoriteCount", IntegerType(), True),
    StructField("CloseDate", TimestampType(), True),
])

# Schema for users table
users_schema = StructType([
    StructField("Id", IntegerType(), False), # unique identifier of each user and cannot be null
    StructField("Reputation", LongType(), True),
    StructField("CreationDate", TimestampType(), False), #Timestamp of when created
    StructField("DisplayName", StringType(), True),
    StructField("LastAccessDate", TimestampType(), True),
    StructField("AboutMe", StringType(), True),
    StructField("Views", LongType(), True),
    StructField("UpVotes", LongType(), True),
    StructField("DownVotes", LongType(), True),
])


#### Subtask 2: implementing two helper functions
Next, we need to implement two helper functions:
1. 'load_csv' that as input argument receives path for a CSV file and a schema and loads the CSV pointed by the path into a Spark DataFrame and returns the DataFrame;
2. 'save_df' receives a Spark DataFrame and saves it as a Parquet file on DBFS.

Note that the column separator in CSV files is TAB character ('\t') and the first row includes the name of the columns. 

BTW, DBFS is the name of the distributed filesystem used by Databricks Community Edition to store and access data.

In [0]:
def load_csv(source_file: "path for the CSV file to load", schema: "schema for the CSV file being loaded as a DataFrame") -> DataFrame:
    return spark.read.csv(source_file, schema = schema, sep = '\t', header = True) 

def save_df(df: "DataFrame to be saved", table_name: "name under which the DataFrame will be saved") -> None:
    path = f"dbfs:/user/hive/warehouse/{table_name}.parquet" #Define the path as variable first for cleaner code
    df.write.mode('overwrite').parquet(path)


In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully. Moreover, at the end there should be four Parquet files named 'badges', 'comments', 'posts', and 'users' in '/user/hive/warehouse'.

Note that we assumed that the data for the project has already been stored on DBFS on the '/FileStore/tables/' path. (I mean as 'badges_csv.gz', 'comments_csv.gz', 'posts_csv.gz', and 'users_csv.gz'.)

In [0]:
%%unittest_main
class TestTask1(unittest.TestCase):
   
    # test 1
    def test_load_badges(self):
        result = load_csv(source_file="/FileStore/tables/badges_csv.gz", schema=badges_schema)
        self.assertIsNotNone(result, "Badges dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 105640, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['UserId', 'Name', 'Date', 'Class']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 2
    def test_load_posts(self):
        result = load_csv(source_file="/FileStore/tables/posts_csv.gz", schema=posts_schema)
        self.assertIsNotNone(result, "Posts dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 61432, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'ParentId', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
                                    'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
                                    'CloseDate']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 3
    def test_load_comments(self):
        result = load_csv(source_file="/FileStore/tables/comments_csv.gz", schema=comments_schema)
        self.assertIsNotNone(result, "Comments dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 58735, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['PostId', 'Score', 'Text', 'CreationDate', 'UserId']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 4
    def test_load_users(self):
        result = load_csv(source_file="/FileStore/tables/users_csv.gz", schema=users_schema)
        self.assertIsNotNone(result, "Users dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 91616, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'AboutMe',
                                    'Views', 'UpVotes', 'DownVotes']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    # test 5
    def test_save_dfs(self):
        dfs = [("/FileStore/tables/users_csv.gz", users_schema, "users"),
               ("/FileStore/tables/badges_csv.gz", badges_schema, "badges"),
               ("/FileStore/tables/comments_csv.gz", comments_schema, "comments"),
               ("/FileStore/tables/posts_csv.gz", posts_schema, "posts")
               ]

        for i in dfs:
            df = load_csv(source_file=i[0], schema=i[1])
            save_df(df, i[2])

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 19.356s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'Spark Application', 'SparkSession', 'Transformations', 'Action', and 'Lazy Evaluation' mean in the context of Spark?

Write your descriptions in the next cell.

# Answers Subtask 4:

### Spark Application:
A Spark application is a type of distributed computing program (An application that can run on multiple computers within a network at the same time), where it's main objective is to process big data. There are two primary components in the Spark application that help manage and execute these distributed data processing tasks, namely, the Driver program and the Executors. These components interact within a cluster, which is managed by a cluster manager. The driver is the "brain" of the application, and could be thought of as running the "main()" function where it manages the execution flow. It's objective is to translate users commands into tasks that are then distributed to various executors. Continuing the analogy of the human body the executors would be the muscles, performing the actions in the form of running computations and storing the intermediate results. These work in conjunction to process large datasets in parallel, for efficient processing. 


### SparkSession:
 A SparkSession is a single entry point to interact with all the functionalities of Apache Spark. In other words, it can be described as an "all-in-one" API which can be used to program in a manner whereby Spark performs all operations without explicitly swithcing contexts. This allowows for numerous ways of handling data, some being; defining schemas, reading from different sources, and performing SQL-like operations.


### Transformations, Action and Lazy Evaluation: 
These 3 concepts are instrinsically interwined so I will will deal with their explanation as one longer paragraph:
(I read a good cooking analogy of these concepts so I'll try to employ it in my explanation as well)

 If we view Spark as a kitchen, we can look at transformations as preparing ingredients, i.e. plans for how to process your data. However, in the Spark kitchen we don't want to rush the meal (fine dining) which is where lazy evaluation comes in. It waits, stacking up the transformations, since it's smarter to figure out the best approach to cooking before firing up the stove. Then, when an order arrives, Spark get's cooking and the chefs spring into action (both food wise and Spark wise), executing all the noted transformations in the most efficient order and manner. The result being the final dataset, ready for consumption (analysis).